In [16]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.models import load_model
from google.colab import drive

Conexión del drive:

In [17]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
%cd /content/drive/MyDrive/CNN/CIFAR100

/content/drive/MyDrive/CNN/CIFAR100


Carga de la data preprocesada:

In [19]:
#Cargar el dataset
data = np.load('data/data.npz')

train_images = data['trainX']
train_labels = data['trainY']
test_images = data['testX']
test_labels = data['testY']

División del conjunto de entrenamiento para obtener conjunto de validación

In [20]:
train_images,val_images,train_labels,val_labels = train_test_split(train_images, train_labels, test_size = 0.1)

In [23]:
print(train_images.shape, val_images.shape, test_images.shape)
print(train_labels.shape, val_labels.shape, test_labels.shape)

(45000, 32, 32, 3) (5000, 32, 32, 3) (10000, 32, 32, 3)
(45000, 100) (5000, 100) (10000, 100)


Ampliación de la data:

        Como se observó en el preprocesamiento de la data, el conjunto de datos de entrenamiento tiene 500 imágenes por clase y que no son suficientes para entrenar adecuadamente al modelo. Es por esto que se utiiza un generador de imágenes que permite aumentar la data realizando modificaciones al conjunto de datos ya existente, como por ejemplo, rotando, desplazando o cortando las imágenes segun se le indique al generador. 

In [24]:
train_datagen = ImageDataGenerator(
        rotation_range = 10,
        zoom_range = 0.1,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        shear_range = 0.1,
        horizontal_flip = True,
        vertical_flip = False,
        )
train_datagen.fit(train_images)

Definir función para la programación de la tasa de aprendizaje:

In [25]:
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    if epoch % 2 == 0:
      return lr * 0.1
    else:
      return lr

callback_lr = LearningRateScheduler(scheduler)

Cargar el modelo base:

In [26]:
model = load_model("models/mobilenet_model.h5")
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 up_sampling2d_1 (UpSamplin  (None, 224, 224, 3)       0         
 g2D)                                                            
                                                                 
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)        3228864   
 onal)                                                           
                                                                 
 global_max_pooling2d_1 (Gl  (None, 1024)              0         
 obalMaxPooling2D)                                               
                                                                 
 dense_1 (Dense)             (None, 100)               102500    
                                                                 
Total params: 3331364 (12.71 MB)
Trainable params: 3309476 (12.62 MB)
Non-trainable params: 21888 (85.50 KB)
___________

In [29]:
model_name = 'mobilenet_best.h5'

checkpoint_filepath = 'trained_models/' + model_name
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model = load_model("models/mobilenet_model.h5")
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

Entrenamiento:

In [30]:
history = model.fit(
    train_datagen.flow(train_images, train_labels, batch_size = 128),
    validation_data = (val_images, val_labels),
    epochs = 20,
    verbose = 1,
    callbacks = [model_checkpoint_callback, callback_lr]
    )
np.save('training_history_mobilenet.npy', history.history)

Epoch 1/20
352/352 [==============================] - 176s 458ms/step - loss: 3.9290 - accuracy: 0.2922 - val_loss: 4.0294 - val_accuracy: 0.3050 - lr: 0.0010
Epoch 2/20
352/352 [==============================] - 158s 449ms/step - loss: 1.7960 - accuracy: 0.5397 - val_loss: 2.0953 - val_accuracy: 0.4906 - lr: 0.0010
Epoch 3/20
352/352 [==============================] - 158s 449ms/step - loss: 1.3005 - accuracy: 0.6380 - val_loss: 1.7965 - val_accuracy: 0.5578 - lr: 0.0010
Epoch 4/20
352/352 [==============================] - 158s 449ms/step - loss: 1.0760 - accuracy: 0.6912 - val_loss: 1.7423 - val_accuracy: 0.5678 - lr: 0.0010
Epoch 5/20
352/352 [==============================] - 163s 463ms/step - loss: 0.9207 - accuracy: 0.7300 - val_loss: 1.6239 - val_accuracy: 0.5952 - lr: 0.0010
Epoch 6/20
352/352 [==============================] - 163s 463ms/step - loss: 0.8205 - accuracy: 0.7565 - val_loss: 1.6344 - val_accuracy: 0.6046 - lr: 0.0010
Epoch 7/20
352/352 [==========================

In [31]:
model = load_model("trained_models/mobilenet_best.h5")
_, acc = model.evaluate(test_images, test_labels)
print(acc * 100.0)

313/313 [==============================] - 12s 37ms/step - loss: 0.9493 - accuracy: 0.7659
76.59000158309937
